<a href="https://colab.research.google.com/github/bisil2/AI1_Final_Project/blob/main/01)_ImagePreprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 01) 이미지 전처리

## 라이브러리 설치 및 Import

In [ ]:
'''SAM 설치 및 패키지 설치'''
# pip install git+https://~는 GitHub에서 직접 Python 패키지를 설치할 때 사용
# 참고 : https://github.com/facebookresearch/segment-anything
!pip install git+https://github.com/facebookresearch/segment-anything.git
!pip install opencv-python pycocotools matplotlib onnxruntime onnx torch

# =============================
# segment-anything : Meta의 Segment Anything 모델 설치
# opencv-python : 이미지 처리 (자르기, 색 변환 등)
# pycocotools : 마스크와 박스 등 COCO 형식 처리
# matplotlib : 결과 이미지 시각화
# onnx, onnxruntime : 모델 경량화
# torch : 모델 실행, 학습, 추론에 필수적인 PyTorch 프레임워크
# =============================

  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-jwjhz4gy
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-jwjhz4gy
  Resolved https://github.com/facebookresearch/segment-anything.git to commit dca509fe793f601edb92606367a655c15ac00fdf
  Preparing metadata (setup.py) ... done
  Created wheel for segment_anything: filename=segment_anything-1.0-py3-none-any.whl size=36592 sha256=c732aa34775e39e6f1480a6fb4e950d654eb1674d85d1ce9243511d7ee432239
  Stored in directory: /tmp/pip-ephem-wheel-cache-5m_ckyr1/wheels/15/d7/bd/05f5f23b7dcbe70cbc6783b06f12143b0cf1a5da5c7b52dcc5
Successfully built segment_anything
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 66.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 67.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
'''SAM 가중치 다운로드'''
!wget -q https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth
CHECKPOINT_PATH = '/content/sam_vit_h_4b8939.pth'

In [ ]:
'''Google Drive 연동'''
from google.colab import drive
drive.mount('/content/drive')

'''필수 라이브러리 import'''
import pandas as pd
import shutil
import zipfile
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import torch
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator

Mounted at /content/drive


## 데이터 불러오기

In [ ]:
''' 메소드 정의 '''

# =============================
# 목적 : Colab 환경의 특정 폴더를 zip으로 압축하여 저장. 백업 용도.
# 매개변수
#  - target : 압축할 폴더 이름
# =============================
def ZipFolder(target):
  # 압축할 폴더와 압축 파일 경로 설정
  rootPath = '/content/'+target
  targetPath = '/content/'+target+'.zip'
  # zipfile.ZipFile로 압축 파일을 열고,
  with zipfile.ZipFile(targetPath, 'w', zipfile.ZIP_DEFLATED) as zipf:
      # os.walk로 폴더 내의 모든 파일을 탐색하고,
      for root, _, files in os.walk(rootPath):
          for file in files:
              # 파일이 있는 상위 경로 + 파일명
              filePath = os.path.join(root, file)
              # 기준 폴더(rootPath)를 기준으로 상대 경로 파악
              arcname = os.path.relpath(filePath, rootPath)
              # 파일(filePath)을 상대경로(arcname)로 알집에 추가
              zipf.write(filePath, arcname)

In [ ]:
'''Label 데이터 불러오기'''
rawdata = pd.read_csv("/content/drive/MyDrive/data/rawdata.csv")

In [ ]:
'''Image 데이터 불러오기'''
zipName = 'rawdata'

# data 폴더 생성
targetPath = '/content/data/'
os.makedirs(targetPath, exist_ok=True)

# 압축 파일 content로 복사 => content에 있으면 처리속도가 비교적 빠름
rootZip = f'/content/drive/MyDrive/data/{zipName}.zip'
targetZip = f'/content/{zipName}.zip'
shutil.copyfile(rootZip, targetZip)

# zipfile.ZipFile로 압축 파일을 열고, 압축된 모든 파일을 targetPath로 이동(압축 해제)
with zipfile.ZipFile(targetZip, 'r') as zip_ref:
  zip_ref.extractall(targetPath)

## 모델 로드 및 설정

In [ ]:
''' 모델 로드 및 설정 '''

# 디바이스 설정 (GPU가 있으면 GPU 사용, 없으면 CPU)
# PyTorch는 모델 연산을 GPU 또는 CPU에서 수행 가능 >> 이미지는 사실상 GPU 원툴
# torch.cuda.is_available()는 GPU가 있는 환경인지 확인
# GPU가 있다면 'cuda:0', 없으면 'cpu' 선택
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# vit_b = Base, vit_l = Large, vit_h = High 중 선택 가능
MODEL_TYPE = "vit_h"

# SAM 모델 로드 >> to(device=Device)를 이용해 모델을 GPU로 올림
sam = sam_model_registry[MODEL_TYPE](checkpoint=CHECKPOINT_PATH).to(device=DEVICE)

# 자동 마스크 생성기 초기화
mask_generator = SamAutomaticMaskGenerator(
    sam, # 사용할 모델 인스턴스
    points_per_side=64, # 한 변에 n개의 포인터를 균일하게 뿌림 (64개 >> 총 64*64 = 4,096 포인트 기준으로 마스크 생성, 배치랑 비슷한 개념?)
    pred_iou_thresh=0.80, # IOU 예측값 파악, 기준치 이상만 유효 마스크로 간주
    stability_score_thresh=0.85, # 생성된 마스크의 품질 안정도가 기준치 이상일 때만 간주
    min_mask_region_area=50 # 최소 면적이 기준치 이상인 마스크만 간주
)

# 모델 실행

In [ ]:
''' 배경 삭제 후, 저장'''

# 100개 단위로 반복 처리 (중간중간 세이브 포인트)
limit = len(rawdata)
start = 0
end = 100

while True:
  # data 폴더 생성
  os.makedirs("data_result", exist_ok=True)

  for j in range(start,end):
    # rawdata의 j번째 행을 row에 저장
    row = rawdata.iloc[j]

    # 이미지 경로 설정(row['image'] = 파일명)
    IMAGE_PATH = f"/content/data/{row['image']}"

    # image 불러오기
    image = cv2.imread(IMAGE_PATH)

    ''' 전처리 '''
    ''' image crop '''
    # BBOX 좌표 기준으로 image crop
    crop_image = image[int(row['ytl']):int(row['ybr']), int(row['xtl']):int(row['xbr'])]

    ''' BGR -> RGB '''
    # CV2는 기본적으로 BGR로 불러오는데, 대부분의 라이브러리는 RGB 형태이므로 변환 필요
    image_rgb = cv2.cvtColor(crop_image, cv2.COLOR_BGR2RGB)

    ''' 채도 증가 '''
    # RGB -> HSV(색상[:,:,0], 채도[:,:,1], 명도[:,:,2])
    hsv = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2HSV)
    # 채도를 1.5배 증가(최솟값 0, 최댓값 255 설정)
    hsv[:,:,1] = np.clip(hsv[:,:,1] * 1.5, 0, 255)
    # HSV -> RGB
    crop_region = cv2.cvtColor(hsv, cv2.COLOR_HSV2RGB)

    ''' 명암 대비 강조 '''
    # CLAHE 객체 생성
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    # 빈 배열 생성
    crop_region_clahe = np.zeros_like(crop_region)
    # RGB 각 채널에 별도로 적용(clahe.apply()는 1채널에만 적용 가능)
    for i in range(3):
        crop_region_clahe[:,:,i] = clahe.apply(crop_region[:,:,i])
    # 변경 사항 저장
    crop_region = crop_region_clahe

    '''세그멘테이션 마스크 생성'''
    # SAM 실행
    output_masks = mask_generator.generate(crop_region)

    # 면적이 가장 큰 마스크 선택
    largest_mask = max(output_masks, key=lambda x: x['area'])
    mask = largest_mask['segmentation']

    # crop된 원본 이미지 복사
    image_with_mask = crop_image.copy()
    # 마스크가 없는 부분(배경)을 검정색으로 설정 (흰색은 질병 영역과 오인할 수도?)
    image_with_mask[~mask] = [0, 0, 0]

    ''' 결과 저장 '''
    # RGB -> BGR (cv2 특)
    # .jpg로 통일
    cv2.imwrite(f"/content/data_result/{row['image'].split('.')[0]}.jpg", cv2.cvtColor(image_with_mask, cv2.COLOR_RGB2BGR))

    # 진행결과 출력
    print(f"{j}/{end}")

  # data_result 폴더 압축
  ZipFolder("data_result")
  # 압축파일 drive에 저장(최대 100개 단위)
  shutil.move(f"/content/data_result.zip", f"/content/drive/MyDrive/data_{start}~{end}.zip")

  # 시작지점, 종료지점 변경
  start += 100
  end += 100
  if end >= limit:
    end = limit
  if start >= limit:
    break

  # 폴더 삭제 (압축 파일이 누적(?)되지 않도록)
  shutil.rmtree('/content/data_result')

KeyboardInterrupt: 